In [13]:
import os
import dotenv
dotenv.load_dotenv()
pinecone_api_key = os.environ.get('PINECONE_API_KEY')
pinecone_region = os.environ.get('PINECONE_REGION')
print(pinecone_api_key)

38cbdebd-ea39-4307-adcd-168b55569cde


In [2]:
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyMuPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import CTransformers

/Users/dilshadahmad/GenAI/medical_chatbot/env/lib/python3.12/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [14]:
# Extract the data from pdf
def load_pdf(data):
    loader = DirectoryLoader(data, glob='*.pdf', loader_cls=PyMuPDFLoader)
    documents = loader.load()
    return documents